In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-08-19 19:32:10--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.15MB/s    in 0.2s    

2021-08-19 19:32:10 (5.15 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon_Book_Reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep='\t', header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [5]:
# Show columns
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [6]:
# Check data types
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# Change data types of selected variables
df = df.withColumn("customer_id",df.customer_id.cast('int'))
df = df.withColumn("product_parent",df.product_parent.cast('int'))
df = df.withColumn("review_date",df.review_date.cast('date'))

In [8]:
# Check data types after changing
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
# Count of all records/reviews
df.count()

3105520

In [10]:
# Drop any N/A values
df.dropna()
df.count()

3105520

In [11]:
# Drop any duplicates
df.dropDuplicates()
df.count()

3105520

In [12]:
# Create review_id_table
reviews_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
reviews_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979| 2005-10-14|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234| 2005-10-14|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049| 2005-10-14|
| RATOTLA3OF70O|   50732546|0373836635|     348672532| 2005-10-14|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717| 2005-10-14|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341| 2005-10-14|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889| 2005-10-14|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054| 2005-10-14|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133| 2005-10-14|
|R1VKEE2NWSWDRU|   15280864|0446531081|     880645124| 2005-10-14|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [13]:
# Create products table
products_df = df.select(["product_id", "product_title"])
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0385730586|Sisterhood of the...|
|0811828964|The Bad Girl's Gu...|
|1844161560|Eisenhorn (A Warh...|
|0373836635|Colby Conspiracy ...|
|0262181533|The Psychology of...|
|0316769487|The Catcher in th...|
|0805076069|Bait and Switch: ...|
|1581603681|Opening Combinati...|
|0300108834|A Little History ...|
|0446531081|Hour Game (King &...|
+----------+--------------------+
only showing top 10 rows



In [14]:
# Only keep distinct products
distinct_products_df = products_df.distinct()
distinct_products_df.count()

779733

In [15]:
# Show first 10 rows of new df
distinct_products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0029148510|Acts of War: Beha...|
|1844494896|Waiting For Kate ...|
|0451216954|Dark Lover (Black...|
|006056718X|Often Wrong, Neve...|
|0965852768|101 Spooktacular ...|
|141960435X|Flip Flop Spanish...|
|0471282502|Experimental Orga...|
|0756613647|Universe: The Def...|
|B0006YGCE8|Seismic risk asse...|
|207030602X|L Etranger (Folio...|
+----------+--------------------+
only showing top 10 rows



In [16]:
# Create customers table
customers_df = df.groupBy("customer_id").count()
# Rename count column
customers_df = customers_df.withColumnRenamed("count", "customer_count")
# Order by customer count to see
customers_df.orderBy(customers_df["customer_count"].desc()).show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50122160|         21922|
|   50732546|          9964|
|   52615377|          2664|
|   45041039|          2215|
|   50776149|          1797|
|   50913245|          1786|
|   20595117|          1643|
|   45273033|          1622|
|   39569598|          1416|
|   49693975|          1384|
+-----------+--------------+
only showing top 10 rows



In [17]:
# Show customers table without orderby
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28735044|             9|
|   15607794|             1|
|   15596972|             1|
|   32371281|             2|
|   12084021|             1|
|   14462635|             1|
|   41052840|             5|
|   52068296|           105|
|   51739623|             1|
|   14544320|             1|
+-----------+--------------+
only showing top 10 rows



Connect to RDS:

In [18]:
# Configure settings for RDS
mode = "append"
jdbc_url="<connection string>:5432/amazon_reviews_db"
config = {"user":"<username>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [19]:
# Write Reviews DataFrame to table
reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [20]:
# Write Customers DataFrame to table
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [21]:
# Write Products DataFrame to table
distinct_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)